In [1]:
import tensorflow as tf
import timeit

In [2]:
w1 = tf.constant([[1.4334732, -1.5244598, 1.139654, 2.723477, 2.372128, -1.8221221],
      [1.6633688, -1.3922757, 2.0349483, 1.6314147, 1.6997916, -1.719175],
      [1.6464833, -1.6136154, 1.6790704, 2.1913328, 1.7154503, -2.122219],
      [2.2029521, -2.2169485, 1.1411709, 1.7363839, 1.9620435, -1.990284],
      [1.864349, -1.9724554, 1.282788, 1.3895663, 1.2881863, -1.3681948],
      [0.4421571, 0.24537054, 0.49080196, -0.0939824, 0.36308903, -0.32526237],
      [-1.6102886, 1.7532632, -1.3683709, -1.2728035, -1.8335032, 1.6637068],
      [-1.0453694, 0.95990705, -1.913037, -1.637573, -1.8312218, 1.9757035],
      [-2.6982157, 1.5073962, -2.243781, -2.7327728, -2.5648139, 1.9095569],
      [-2.0628226, 2.3980575, -1.3550557, -2.1798916, -2.1485612, 2.2912557]], dtype=tf.float32)

w2 = tf.constant([[2.220895], [-2.903738], [1.7675139],
      [2.3042984], [2.6292808], [-2.763858]], dtype=tf.float32)

b1 = tf.constant([-1.6460503, 1.5486399, -1.5155386, -1.6247352, -1.2638505, 1.5515162], dtype=tf.float32)
b2 = tf.constant([-1.1827456], dtype=tf.float32)

x1 = tf.constant([0.85, 0.86, 0.76, 0.73, 0.95, 0.5, 0.5, 0.5, 0.5, 0.5], dtype=tf.float32)
x2 = tf.constant([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.67, 0.87], dtype=tf.float32)


In [21]:
@tf.function
def neural_net(x: tf.Tensor, w1: tf.Tensor, w2: tf.Tensor, b1: tf.Tensor, b2: tf.Tensor) -> tf.Tensor:
    tf.debugging.assert_type(x, tf_type=tf.float32)
    hidden_layer = tf.sigmoid(tf.add(tf.matmul(tf.reshape(x, [1, -1]), w1), b1))
    y = tf.sigmoid(tf.add(tf.matmul(hidden_layer, w2), b2))
    return y


## Results

In [22]:
y1 = neural_net(x1, w1, w2, b1, b2)
y2 = neural_net(x2, w1, w2, b1, b2)
print(f'y1: {y1}')
print(f'y2: {y2}')

y1: [[0.9977336]]
y2: [[0.00667112]]


### With @tf.function annotation

In [9]:
func_y1 = timeit.timeit(lambda: neural_net(x1, w1, w2, b1, b2), number=3)
print(f'Execution time in graph mode for tensor x1: {func_y1*1000} milliseconds')

Execution time in graph mode for tensor x1: 95.6233750000024 milliseconds


In [16]:
func_y2 = timeit.timeit(lambda: neural_net(x2, w1, w2, b1, b2), number=3)
print(f'Execution time in graph mode for tensor x2: {func_y2*1000} milliseconds')

Execution time in graph mode for tensor x2: 100.91687799996407 milliseconds


### Without @tf.function annotation

In [19]:
eager_y1 = timeit.timeit(lambda: neural_net(x1, w1, w2, b1, b2), number=3)
print(f'Execution time in eager mode for tensor x2: {eager_y1*1000} milliseconds')

Execution time in eager mode for tensor x2: 3.1411729999604177 milliseconds


In [20]:
eager_y2 = timeit.timeit(lambda: neural_net(x2, w1, w2, b1, b2), number=3)
print(f'Execution time in eager mode for tensor x2: {eager_y2*1000} milliseconds')

Execution time in eager mode for tensor x2: 4.613655000014205 milliseconds


## Wnioski

Czas odpowiedzi sieci w trybie graph mode wyniósł 95,6 ms dla pierwszego wektora oraz 100,9 ms dla drugiego wektora.

Czas odpowiedzi sieci w trybie eager mode wyniósł 3,14 ms dla pierwszego wektora oraz 4,61 ms dla drugiego wektora.

Znacznie dłuższy czas odpowiedzi sieci w trybie graph mode wynika z potrzeby skonstruowania grafu (tf.Graph), który przyspiesza wykonywanie operacji tensorowych m.in. poprzez ich zrównoleglenie. W przypadku kilku prostych operacji, czas potrzebny do skonstruowania grafu przewyższa czas samych obliczeń. Taka sytuacja ma miejsce jedynie przy pierwszym wywołaniu funkcji. Powyższe odpytania sieci miały każdorazowo miejsce bezpośrednio po jej deklaracji. Gdyby wywołać funkcję ponownie w trybie graph mode, czas odpowiedzi zmaleje. 